# Assignment 3 - Building a Custom Visualization<br>

In this assignment you must choose **one** of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use `matplotlib` so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.  
<br>
Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented Task-driven Visualizations: Allowing Users to Make Better, More Confident Decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
<br>
In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. 
<br>
[Accompanying video](https://www.youtube.com/watch?v=BI7GAs-va-Q)  
<br>
In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number of votes in a given district for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="Assignment3Fig1.png" alt="Figure 1" style="width: 500px; align: center;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from Ferreira et al. 2014.</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="Assignment3Fig2c.png" alt="Figure 2c" style="width: 500px; align: center;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from Ferreira et al. 2014.<br><br> Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with at least three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.

**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [1]:
%matplotlib tk

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact, Checkbox, Text

def monte_carlo_prob_est(sample_mean, sampling_dist_std, lower_bound, upper_bound):
    iterations = 10000
    means = np.random.normal(sample_mean, sampling_dist_std, iterations) ## Sampling the sample mean distribution
                                                                         ## represented by the 95% confidence interval
                                                                         ## error bars on the barchart (Using the distribution's STD here)
    within_range = means[(means > lower_bound) & (means < upper_bound)]  ## Returning which sample means are within the user-defined range
    
    frac_within_range = float(len(within_range)) / iterations  ## Approximating the probability that the true mean is within the
                                                               ## user-defined range via the fraction of sample means that lie within the range
    return frac_within_range
    
    

def on_click(event):
    bound = event.ydata
    bounds.append(bound)
    
    if len(bounds) > 2:
        del bounds[0:2]
        [line.remove() for line in ax.lines[2:]]
        [collection.remove() for collection in ax.collections[1:]]
        [bar.set_facecolor('white') for bar in bars]
    
    ax.axhline(y = bound, linestyle='-', linewidth=2, color='xkcd:gunmetal')
    
    if len(bounds) == 1:
        print('Selected range boundary: y = {: .3f}'.format(bound))
        
    elif len(bounds) == 2:
        ax.fill_between(ax.get_xlim(), bounds[0], bounds[1], color='xkcd:gunmetal', alpha=0.3)
        print('Selected range boundary: y = {: .3f}'.format(bound), '\n')
        
        print('The approximate probabilities that the true means lie within the given range: \n')
        
        for bar, year, mean, std in zip(bars, df.index, df['Sample Mean'], df['Sample STD']):
            probability = (monte_carlo_prob_est(mean, std/np.sqrt(sample_size), min(bounds), max(bounds)))
            bar.set_facecolor(cmap(probability))
            print('{}: p ~ {:.3f}'.format(year, probability))
            
        print('\n~~~~~~~~~~~~~~\n\n')
           
    fig.canvas.draw() #If this is not included, the lines will be drawn only when the callback is disconnected
    
    return


def prob_true_mean_in_range(lb, ub, use_textboxes):
    if len(ax.lines) > 2:
        [line.remove() for line in ax.lines[2:]]
        [bar.set_facecolor('white') for bar in bars]
        
    if len(ax.collections) > 1:
        [collection.remove() for collection in ax.collections[1:]]
    
    if use_textboxes and (float(lb) != 0. or float(ub) != 0.):       
        if float(lb) > float(ub):
            raise ValueError('Lower bound is greater than upper bound!')
        
        ax.axhline(y = float(lb), linestyle='-', linewidth=2, color='xkcd:gunmetal')
        ax.axhline(y = float(ub), linestyle='-', linewidth=2, color='xkcd:gunmetal')
        
        ax.fill_between(ax.get_xlim(), float(lb), float(ub), color='xkcd:gunmetal', alpha=0.3)
        
        print('The approximate probabilities that the true means lie within the given range: \n')
        
        for bar, year, mean, std in zip(bars, df.index, df['Sample Mean'], df['Sample STD']):
            probability = (monte_carlo_prob_est(mean, std/np.sqrt(sample_size), float(lb), float(ub)))
            bar.set_facecolor(cmap(probability))
            print('{}: p ~ {:.3f}'.format(year, probability))
            
        print('\n~~~~~~~~~~~~~~\n')
        
    elif use_textboxes == False:
        global bounds
        bounds = []
        
        print('- Click on the interactive plot to choose your "y" bounds - \n')
        
        fig.canvas.mpl_connect('button_press_event', on_click)
    
    return

# Use the following data for this assignment:
#################################################

np.random.seed(12345)

sample_size = 3650

df = pd.DataFrame([np.random.normal(32000, 200000, sample_size), 
                   np.random.normal(43000, 100000, sample_size), 
                   np.random.normal(43500, 140000, sample_size), 
                   np.random.normal(48000, 70000, sample_size)], 
                   index=[1992, 1993, 1994, 1995])

################################################

df['Sample Mean'] = df.apply(np.mean, axis=1)
df['Sample STD']  = df.apply(np.std, axis=1)

lower_bound = Text(value=str(0), description='Lower Bound:')
upper_bound = Text(value=str(0), description='Upper Bound:')
use_textboxes = Checkbox(value=True, description='Use Textboxes?')

fig, ax = plt.subplots(figsize=(8,6))

## yerr below gives the 95% confidence interval: xbar +/- Z(sigma/sqrt(N))
bars = ax.bar(df.index.to_numpy(), df['Sample Mean'], yerr=df['Sample STD'] * (1.96 / np.sqrt(sample_size)), 
              color='white', edgecolor='black', linewidth=1.5, error_kw={'elinewidth': 1.5, 'capsize': 15, 'capthick': 1.5},
              label='Sample mean with 95% CI')

cmap = mpl.cm.tab20c_r
norm = mpl.colors.Normalize(vmin=0., vmax=1.)

cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, location='right', ticks=np.arange(0., 1.1, 0.1))
cbar.ax.tick_params(width=1., labelsize=11)
cbar.set_label('Probability', rotation=270, labelpad=20, fontsize=16)

bar_width = ax.patches[0].get_width() #All bars (patches.Rectangle) have the same width

ax.set_xlim([df.index[0]-(2./3.)*bar_width, df.index[-1]+(2./3.)*bar_width])
ax.set_ylim([0., round(df['Sample Mean'].max()*1.2, -3)])
ax.set_xticks(df.index.to_numpy())
ax.get_yaxis().set_major_formatter(mpl.ticker.FuncFormatter(lambda y, p: format(int(y), ',')))
ax.minorticks_on()
ax.tick_params(which='minor', length=4, width=1.0, bottom=False)
ax.tick_params(which='major', length=8, width=1.5, labelsize=13)
ax.set_xlabel('Year', fontsize=16, labelpad=10)
ax.set_ylabel('Data', fontsize=16, labelpad=10)
ax.set_title('Probability True Mean Lies Within User-Defined Range', fontsize=18, pad=12)

ax.legend(loc='upper left')

ax.annotate('(c) Tim Gburek', [0.79, 0.1], xycoords='figure fraction', fontsize=8)

fig.tight_layout()

# print('Base Artists Before Annotation:\n')
# print('{:<12}'.format('Bars: '), ax.patches)
# print('{:<12}'.format('Error Bars: '), ax.collections)
# print('{:<12}'.format('Caps: '), ax.lines)

interact(prob_true_mean_in_range, lb=lower_bound, ub=upper_bound, use_textboxes=use_textboxes);


interactive(children=(Text(value='0', description='Lower Bound:'), Text(value='0', description='Upper Bound:')…